# In this notebook
This notebook runs the code from `create_masks.py` in the notebook. Turns out that
this code runs quickly enough that this is no problem.

In [1]:
import os
import argparse

from PIL import Image, ImageDraw

import pandas as pd

In [2]:
dump_path = '/scratch/zach/masks_raw/'

In [4]:
# Get list of masks that we want to generate:
fresno_dir = "/home/zdc6/data/SolarPV/fresno/"
files = list(filter(lambda x: x.split(".")[-1] == "tif", os.listdir(fresno_dir)))
print(f"{len(files)} files to read...")
# Get references
image_map = pd.read_csv(
	"/home/zdc6/data/SolarPV/polygonDataExceptVertices.csv", index_col=0
).set_index("image_name")

polygon_vertices = pd.read_csv(
	"/home/zdc6/data/SolarPV/polygonVertices_PixelCoordinates.csv"
).set_index("polygon_id")

412 files to read...


In [ ]:
key_errors = []
attr_errors = []
# Iterate through each image.
count = 0
polygon_count = 0
for fn in files:
	print(f"Reading file {fn}...")
	image_name = fn.split(".")[0]
	# Get polygons for this image.
	try:
		polygons = image_map.loc[image_name]
	except KeyError:
		print(f"No polygons found for image {image_name}")
		count += 1
		continue

	# Create new blank image and drawing
	new_img = Image.new(str(1), (5000, 5000), color=0)
	mask = ImageDraw.Draw(new_img)

	# Loop through all of the polygons:
	try:
		for idx, polygon in polygons.iterrows():
			try:
				vertices = polygon_vertices.loc[int(polygon["polygon_id"])]
				polygon_count += 1
			except KeyError:
				key_errors.append(fn)
				continue
			vertices = [i for i in vertices[1:] if not (pd.isna(i))]
			mask.polygon(vertices, fill=1)
	except AttributeError:
		attr_errors.append(fn)
		# If there is no attribute, don't iterate through
		vertices = polygon_vertices.loc[int(polygons["polygon_id"])]
		polygon_count += 1
		vertices = [i for i in vertices[1:] if not (pd.isna(i))]
		mask.polygon(vertices, fill=1)

	print("Saving image...")
	count += 1
	new_img.save(dump_path + fn)